In [ ]:
### Colab define cell
FOLDER_PATH = '/content/gdrive/MyDrive/Counterfactual-prototype-main/LORE/LORE-master'

try:
    import google.colab
    RUNNING_ON_COLAB = True
except: 
    RUNNING_ON_COLAB = False

if RUNNING_ON_COLAB: 
    import sys
    from google.colab import drive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials 
    drive.mount('/content/gdrive', force_remount=True)
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    gdrive = GoogleDrive(gauth)
    sys.path.append(FOLDER_PATH)

In [1]:
from utils.prediction import PredictionTypeWrapper, PredictionTypeWrapper, generate_local_predictions, wrap_information
from dice import DiCECounterfactaulWrapper, DiCESingleNeuronOutputWrapper
from utils.cf import generate_cf_for_all
from utils import load

import tensorflow as tf
import dice_ml
import numpy as np
import pandas as pd

### Set random seed
seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)

In [2]:
### Load data
dataset = load.SelectableDataset.Diabetes
data, balanced_data, X, Y, encoder, scaler, n_features, n_classes, feature_names, target_name = load.load_dataset(dataset)
X_train, Y_train, X_test, Y_test, X_validation, Y_validation = load.load_training_data(dataset)
model = load.load_trained_model_for_dataset(dataset)

Loaded model from disk


In [3]:

diabetes_feature_range = (X_train.min(axis=0), X_train.max(axis=0))
# store all information 
local_data_dict = generate_local_predictions( X_test, Y_test, model, scaler, encoder )
# sorting by different conditions
true_positives,true_negatives, false_positives, false_negatives = wrap_information( local_data_dict )
# get all prediction types
all_predictions = PredictionTypeWrapper(true_positives,true_negatives, false_positives, false_negatives)

In [4]:
# Dataset for training an ML model
temp_df = pd.DataFrame(X, columns=feature_names)
temp_df[target_name] = Y[:, 1]
d = dice_ml.Data(dataframe=balanced_data, continuous_features=feature_names, outcome_name=target_name)
wrapped_model = DiCESingleNeuronOutputWrapper(model)
m = dice_ml.Model(model=wrapped_model, backend="TF2")
exp = dice_ml.Dice(d,m)

In [5]:
### Generate single counterfactual
dice_wrapper = DiCECounterfactaulWrapper(exp, feature_names)
dice_wrapper.run_counterfactual_print_result(all_predictions.get_true_negative(3))

Diverse Counterfactuals found! total time taken: 00 min 53 sec


{'index': 12,
 'original_vector': [3.0, 116.0, 69.147541, 15.0, 105.0, 26.3, 0.100066, 24.0],
 'scaled_vector': [0.1764705882352941,
  0.5829145728643216,
  0.6065573770491803,
  0.15151515151515152,
  0.12411347517730496,
  0.39195230998509695,
  0.009421841541755885,
  0.05882352941176472],
 'ground_truth': 0,
 'predictions': 0,
 'prediction_type': 'TRUE NEGATIVE',
 'cf': [3.0, 117.0, 69.0, 18.0, 491.0, 26.7, 0.108, 44.0],
 'time': 53.240095138549805}

In [8]:
## Generate counterfactuals for all
dice_cf_df = generate_cf_for_all(
    all_predictions,
    dice_wrapper.run_counterfactual_print_result,
    feature_names
    )


====================Doing TruePositive====================
|  

====================Running...====================
| Instance 0 


KeyboardInterrupt: 

In [ ]:
dice_cf_df.to_csv('./dice/cf_result.csv')